In [7]:
import pandas as pd
import numpy as np
import seaborn as sns

In [41]:
a1 = pd.read_csv("computed_data.csv")

In [10]:
a1.head()

,month,town,flat_type,block,street_name,storey_range,floor_area_sqm,flat_model,lease_commence_date,resale_price,remaining_lease
0,2000-01,ANG MO KIO,3 ROOM,170,ANG MO KIO AVE 4,07 TO 09,69.0,Improved,1986,147000.0,0
1,2000-01,ANG MO KIO,3 ROOM,174,ANG MO KIO AVE 4,04 TO 06,61.0,Improved,1986,144000.0,0
2,2000-01,ANG MO KIO,3 ROOM,216,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,159000.0,0
3,2000-01,ANG MO KIO,3 ROOM,215,ANG MO KIO AVE 1,07 TO 09,73.0,New Generation,1976,167000.0,0
4,2000-01,ANG MO KIO,3 ROOM,218,ANG MO KIO AVE 1,07 TO 09,67.0,New Generation,1976,163000.0,0


In [11]:
a1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914357 entries, 0 to 914356
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                914357 non-null  object 
 1   town                 914357 non-null  object 
 2   flat_type            914357 non-null  object 
 3   block                914357 non-null  object 
 4   street_name          914357 non-null  object 
 5   storey_range         914357 non-null  object 
 6   floor_area_sqm       914357 non-null  float64
 7   flat_model           914357 non-null  object 
 8   lease_commence_date  914357 non-null  int64  
 9   resale_price         914357 non-null  float64
 10  remaining_lease      914357 non-null  int64  
dtypes: float64(2), int64(2), object(7)
memory usage: 76.7+ MB


In [12]:
a1.isnull().sum()

month                  0
town                   0
flat_type              0
block                  0
street_name            0
storey_range           0
floor_area_sqm         0
flat_model             0
lease_commence_date    0
resale_price           0
remaining_lease        0
dtype: int64

In [13]:
for value in a1.columns:
    if a1[value].dtypes == "object":
        a1[value] = pd.Categorical(a1[value]).codes


In [14]:
a1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 914357 entries, 0 to 914356
Data columns (total 11 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   month                914357 non-null  int16  
 1   town                 914357 non-null  int8   
 2   flat_type            914357 non-null  int8   
 3   block                914357 non-null  int16  
 4   street_name          914357 non-null  int16  
 5   storey_range         914357 non-null  int8   
 6   floor_area_sqm       914357 non-null  float64
 7   flat_model           914357 non-null  int8   
 8   lease_commence_date  914357 non-null  int64  
 9   resale_price         914357 non-null  float64
 10  remaining_lease      914357 non-null  int64  
dtypes: float64(2), int16(3), int64(2), int8(4)
memory usage: 36.6 MB


In [15]:
X = a1.drop('resale_price', axis = 1)
y = a1.pop('resale_price')

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=.3,random_state=1)

In [18]:
from sklearn.tree import DecisionTreeRegressor

In [19]:
model = DecisionTreeRegressor(random_state=1)

In [20]:
model.fit(X_train,y_train)

DecisionTreeRegressor(random_state=1)

In [21]:
ytest_pred = model.predict(X_test)
ytrain_pred = model.predict(X_train)

In [22]:
from sklearn import metrics
from sklearn.metrics import mean_squared_error, r2_score

In [23]:
metrics.mean_squared_error(ytest_pred,y_test)

895370573.7664614

In [24]:
r2 = r2_score(ytest_pred,y_test)

In [25]:
print(r2)

0.9678973467804083


In [26]:
x_pred = model.predict(X_train)


In [27]:
metrics.mean_squared_error(ytrain_pred,y_train)

4010162.3167806165

In [28]:
r2 = r2_score(ytrain_pred,y_train)
print(r2)

0.9998563492468835


In [29]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import GridSearchCV

In [30]:
param_grid = {
    'max_depth' : [None,5,10,15],
    'min_samples_split' : [2,5,10],
    'min_samples_leaf' : [1,2,4],
    'max_features' : [11]
}

In [31]:
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)


GridSearchCV(cv=5, estimator=DecisionTreeRegressor(random_state=1),
             param_grid={'max_depth': [None, 5, 10, 15], 'max_features': [11],
                         'min_samples_leaf': [1, 2, 4],
                         'min_samples_split': [2, 5, 10]},
             scoring='neg_mean_squared_error')

In [32]:
best_param = grid_search.best_params_

In [33]:
best_dt_regressor = DecisionTreeRegressor(**best_param)
best_dt_regressor.fit(X_train,y_train)

DecisionTreeRegressor(max_features=11, min_samples_leaf=2, min_samples_split=10)

In [34]:
y_pred_dt = best_dt_regressor.predict(X_test)

mse_dt = mean_squared_error(y_test,y_pred_dt)
print("Best decision tree MSE:" , mse_dt)
print("Best decision tree hyperparameters:", best_param)

Best decision tree MSE: 753777168.1975431
Best decision tree hyperparameters: {'max_depth': None, 'max_features': 11, 'min_samples_leaf': 2, 'min_samples_split': 10}


In [35]:
r2_dt = r2_score(y_test,y_pred_dt) # Testng results
print(r2_dt)

0.9730776638121931


In [36]:
y_train_pred_dt = best_dt_regressor.predict(X_train)

In [42]:
r2_train_dt = r2_score(y_train,y_train_pred_dt) # Training results
print(r2_train_dt)

0.9919716470160419


In [46]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV

# Decision Tree Regressor
dtr = DecisionTreeRegressor()

# hyperparameters
param_grid = {
    'max_depth': [2, 5, 10, 15, 20, 22],
    'min_samples_split': [2, 3, 4, 5],
    'min_samples_leaf': [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20],
    'max_features': ['auto', 'sqrt', 'log2']
}

# gridsearchcv
grid_search = GridSearchCV(estimator=dtr, param_grid=param_grid, cv=5)
grid_search.fit(X_train, y_train)
print("Best hyperparameters:", grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# evalution metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(" ")
print('Mean squared error:', mse)
print('Mean Absolute Error', mae)
print('Root Mean squared error:', rmse)
print(" ")
print('R-squared:', r2)

Best hyperparameters: {'max_depth': 22, 'max_features': 'sqrt', 'min_samples_leaf': 2, 'min_samples_split': 4}
 
Mean squared error: 1249486201.6881814
Mean Absolute Error 22396.331782148434
Root Mean squared error: 35348.072107092084
 
R-squared: 0.9553726366317064


In [48]:
# Saving the model
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(best_model, file)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [ ]:
# ['quantity tons_log', 'application', 'thickness_log', 'width','country','customer','product_ref']].values, X_ohe, X_be
new_sample = np.array([[np.log(40), 10, np.log(250), 0, 28,30202938,1670798778,'PL','Won']])
new_sample_ohe = ohe.transform(new_sample[:, [7]]).toarray()
new_sample_be = ohe2.transform(new_sample[:, [8]]).toarray()
new_sample = np.concatenate((new_sample[:, [0,1,2, 3, 4, 5, 6,]], new_sample_ohe, new_sample_be), axis=1)
new_sample1 = scaler.transform(new_sample)
new_pred = best_model.predict(new_sample1)
print('Predicted selling price:', np.exp(new_pred))

#lets try using Random forest

In [44]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [45]:
new_sample = np.array([[8740, 999, np.log(44), 55, np.log(11)]])
new_sample = scaler.transform(new_sample[:, :5])
new_pred = model.predict(new_sample)[0]
np.exp(new_pred)

NotFittedError: This StandardScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
import pickle
with open('model.pkl', 'wb') as file:
    pickle.dump(model, file)
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [40]:
# We know that after One Hot Encoding, the value of Central, East, North, South, and West is one at column 1,2,3,4, and 5, respectively (following alphabetical order).
area = ['Central','East','North','South','West']
pred_price_central = model.predict([[1.0, 0.0, 0.0, 0.0, 0.0, 4, 9, 95.0, 75]])[0]
pred_price_east = model.predict([[0.0, 1.0, 0.0, 0.0, 0.0, 4, 9, 95.0, 75]])[0]
pred_price_north = model.predict([[0.0, 0.0, 1.0, 0.0, 0.0, 4, 9, 95.0, 75]])[0]
pred_price_south = model.predict([[0.0, 0.0, 0.0, 1.0, 0.0, 4, 9, 95.0, 75]])[0]
pred_price_west = model.predict([[0.0, 0.0, 0.0, 0.0, 1.0, 4, 9, 95.0, 75]])[0]
resale_price = [pred_price_central,pred_price_east,pred_price_north,pred_price_south,pred_price_west]
predict_df = pd.DataFrame(list(zip(area, resale_price)),
              columns=['Area','Predicted HDB price (SGD)'])
predict_df.round()

ValueError: X has 9 features, but DecisionTreeRegressor is expecting 10 features as input.